In [5]:
import os
import json

def find_json_files(directory):
    json_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.json'):
                json_files.append(os.path.join(root, file))
    return json_files

def extract_track_ids(json_files):
    track_ids = []
    for json_file in json_files:
        with open(json_file, 'r') as file:
            data = json.load(file)
            albums = data.get('albums', [])
            for album in albums:
                tracks = album.get('tracks', {}).get('items', [])
                print("album: ", album.get('name'), "tracks: ", len(tracks), "tracks: ", tracks)
                for track in tracks:
                    track_ids.append(track.get('id'))
    return track_ids

# Specify the folder path here
folder_path = 'C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\ALBUMS'

# Find all JSON files within the folder and its subfolders
json_files = find_json_files(folder_path)

# Extract all track IDs from these JSON files
track_ids = extract_track_ids(json_files)

unique_track_ids = list(set(track_ids))

# Save unique track IDs to a file
with open('unique_track_ids.txt', 'w') as f:
    for track_id in unique_track_ids:
        f.write(f"{track_id}\n")

print("Unique track IDs have been saved to unique_track_ids.txt.")
